## sqlite3

In [ ]:
import sqlite3
connection = sqlite3.connect('chinook.db')
cursor = connection.cursor()

In [ ]:
# list out tables -- .table does not work from Python sqlite3
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

In [ ]:
# get table information (column names, types, settings)
cursor.execute('PRAGMA table_info(artists);')
cursor.fetchall()

In [ ]:
# SELECT the first 5 rows of artists
cursor.execute('SELECT * FROM artists LIMIT 5;')
cursor.fetchall()

In [ ]:
# especially for longer queries, it helps to format them like this, with each SQL command on a separate line
query = """
SELECT *
FROM artists
LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

In [ ]:
# get table information (column names, types, settings)
cursor.execute('PRAGMA table_info(invoices);')
cursor.fetchall()

In [ ]:
# save table column names in a list
cursor.execute('PRAGMA table_info(invoices);')
results = cursor.fetchall()
column_names = [r[1] for r in results]

In [ ]:
column_names

In [ ]:
cursor.execute('SELECT * FROM invoices LIMIT 5;')
cursor.fetchall()

In [ ]:
# ORDER BY
cursor.execute('SELECT Total, InvoiceDate from invoices ORDER BY Total DESC LIMIT 5;')
cursor.fetchall()

In [ ]:
# WHERE statement
cursor.execute('SELECT Total, BillingCountry from invoices WHERE BillingCountry == "Canada" LIMIT 5;')
cursor.fetchall()

In [ ]:
# WHERE using an inserted argument
cursor.execute('SELECT Total, BillingCountry from invoices WHERE BillingCountry == ? LIMIT 5;', ('Canada',))
cursor.fetchall()

In [ ]:
# LIKE command
cursor.execute('SELECT Total, BillingCountry from invoices WHERE BillingCountry LIKE "%can%" LIMIT 5;')
cursor.fetchall()

In [ ]:
# GROUP BY statement
cursor.execute('SELECT SUM(Total), BillingCountry from invoices GROUP BY BillingCountry ORDER BY SUM(Total) DESC LIMIT 5;')
cursor.fetchall()

In [ ]:
# examine column names for invoice_items table
cursor.execute('PRAGMA table_info(invoice_items);')
cursor.fetchall()

In [ ]:
# examine a sample of the data
cursor.execute('SELECT * FROM invoice_items LIMIT 5;')
cursor.fetchall()

In [ ]:
# aliases can be used to rename columns and tables
# according to some SQL style guides, it's not best practice to alias a table
cursor.execute('SELECT i.TrackID as tid, i.UnitPrice as up FROM invoice_items as i LIMIT 5;')
cursor.fetchall()

In [ ]:
# DISTINCT
cursor.execute('SELECT DISTINCT UnitPrice FROM invoice_items;')
cursor.fetchall()

In [ ]:
# JOIN
# get tracks that were purchased and combine with the country
query = """
SELECT invoices.BillingCountry, invoice_items.TrackId
FROM invoices
JOIN invoice_items
ON invoices.InvoiceId = invoice_items.InvoiceId
LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

In [ ]:
# get number of purchased tracks for each track by country, sorted by the top-most purchased
query = """
SELECT invoice_items.TrackId, COUNT(invoice_items.TrackId), invoices.BillingCountry
FROM invoices
JOIN invoice_items
ON invoices.InvoiceId = invoice_items.InvoiceId
GROUP BY invoices.BillingCountry
ORDER BY COUNT(invoice_items.TrackId) DESC
LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

In [ ]:
# multiple JOINs
query = """
SELECT tracks.Name, COUNT(invoice_items.TrackId), invoices.BillingCountry
FROM invoices
JOIN invoice_items
ON invoices.InvoiceId = invoice_items.InvoiceId
JOIN tracks
ON tracks.TrackId = invoice_items.TrackId
GROUP BY invoices.BillingCountry
ORDER BY COUNT(invoice_items.TrackId) DESC
LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

In [ ]:
# this same command as above can also be done with a subquery like this, but is easier with multiple joins
query = """
SELECT tracks.Name, invoice_merged.track_count, invoice_merged.BillingCountry
FROM
    (SELECT ii.TrackId, COUNT(ii.TrackId) as track_count, i.BillingCountry
    FROM invoices as i
    JOIN invoice_items as ii
    ON i.InvoiceId = ii.InvoiceId
    GROUP BY BillingCountry) as invoice_merged
JOIN tracks
ON tracks.TrackId = invoice_merged.TrackId
ORDER BY track_count DESC
LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

In [ ]:
# be sure to close the connection when done
connection.close()

### Storing data in a sqlite3 database

In [8]:
# hypothetical book sales data
book_data = [('12-1-2020', 'Practical Data Science With Python', 19.99, 1),
       ('12-15-2020', 'Python Machine Learning', 27.99, 1),
       ('12-17-2020', 'Machine Learning For Algorithmic Trading', 34.99, 1)]
print(type(book_data))

<class 'list'>


In [ ]:
# CREATE and INSERT
connection = sqlite3.connect('book_sales.db') #book_sales.db file will be created if it does not exist.
cursor = connection.cursor()

In [ ]:
# Create table
cursor.execute('''CREATE TABLE IF NOT EXISTS book_sales
             (date text, book_title text, price real, quantity real)''')
#table: book_sales, column:date, datatype:text

In [ ]:
# the table is now there
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

In [ ]:
# Insert a row of data
cursor.execute("INSERT INTO book_sales VALUES (?, ?, ?, ?)", book_data[0])
'''We can also use string formatting to place our values into the 
query, but this is not recommended. String formatting of SQL queries is a little less 
safe because we can suffer from a SQL injection attack. For example, if a hacker was 
able to put an arbitrary string into our SQL query, they could insert something like ; 
DROP TABLE book_sales''

In [ ]:
cursor.execute('SELECT * FROM book_sales ;')
cursor.fetchall()

In [ ]:
# Save the changes with .commit()
# Without this line, the inserted data will not be saved in the database after we close the connection
connection.commit()

In [ ]:
# insert several records at a time
cursor.executemany('INSERT INTO book_sales VALUES (?, ?, ?, ?)', book_data[1:])
# don't forget to save the changes
connection.commit()

In [ ]:
cursor.execute('SELECT * FROM book_sales;')
cursor.fetchall()

In [ ]:
connection.close()

## SQLAlchemy

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///book_sales.db')
connection = engine.connect()

In [ ]:
result = connection.execute("select * from book_sales")
result

In [ ]:
list(result)

In [ ]:
for row in result:
    print(row['date'])

In [ ]:
result = connection.execute("select * from book_sales")
for row in result:
    print(row['date'])

In [ ]:
# be sure to close the connection when finished
connection.close()

In [ ]:
# we can also use the with clause to automatically close the connection
with engine.connect() as connection:
    result = connection.execute("select * from book_sales")
    for row in result:
        print(row)

In [ ]:
connection.closed

In [ ]:
# the connection is closed from the 'with' statement, so we can't use it
# notice in the middle and at the bottom of the error, it says 'This Connection is closed'
result = connection.execute("select * from book_sales")

In [ ]:
from sqlalchemy import MetaData, Table

metadata = MetaData(engine)
book_sales = Table('book_sales', metadata, autoload=True)
conn = engine.connect()

In [ ]:
res = conn.execute(book_sales.select())
for r in res:
    print(r)

In [ ]:
ins = book_sales.insert().values(book_title='machine learining', price='10.99')
conn.execute(ins)

In [ ]:
res = conn.execute(book_sales.select())
for r in res:
    print(r)